In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.core.agent.providers import OpenAIAutomataAgent
from automata.core.singletons.dependency_factory import dependency_factory
from automata.core.singletons.py_module_loader import py_module_loader
from automata.core.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

INFO:automata.core.tools.factory:Building dependencies for toolkit_list ['context-oracle']...
INFO:automata.core.tools.factory:Building symbol_doc_embedding_handler...
INFO:automata.core.tools.factory:Creating dependency symbol_doc_embedding_handler
INFO:automata.core.tools.factory:Creating dependency symbol_search
INFO:automata.core.tools.factory:Creating dependency symbol_graph
INFO:automata.core.tools.factory:Creating dependency symbol_code_embedding_handler
INFO:automata.core.tools.factory:Creating dependency embedding_similarity_calculator
INFO:automata.core.tools.factory:Creating dependency py_context_retriever
INFO:automata.core.tools.factory:Building symbol_search...
INFO:automata.core.tools.factory:Building symbol_code_embedding_handler...
INFO:automata.core.tools.factory:Building embedding_similarity_calculator...


In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(7)
    .build()
)

INFO:automata.core.tools.factory:Creating dependency symbol_rank
INFO:automata.core.tools.factory:Creating dependency subgraph
INFO:automata.core.symbol.graph:Pre-computing bounding boxes for all rankable symbols
INFO:automata.core.symbol.graph:Finished pre-computing bounding boxes for all rankable symbols in 8.512240171432495 seconds
INFO:automata.core.symbol.graph:Building the rankable symbol subgraph...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 814/814 [00:05<00:00, 161.18it/s]
INFO:automata.core.symbol.graph:Built the rankable symbol subgraph


In [5]:
# Initialize the agent
# instructions = "Explain how OpenAI is used by the codebase. Note in particularly how it is used in embeddings and how agents consume it"
instructions = "Provide a comprehensive explanation of how SymbolRank and SymbolSearch work. Return your answer as a valid JSON string."
agent = OpenAIAutomataAgent(instructions, config=agent_config)

DEBUG:automata.core.agent.providers:Setting up agent with tools = [OpenAITool(function=<bound method ContextOracleToolkitBuilder._get_context of <automata.core.tools.builders.context_oracle.ContextOracleOpenAIToolkitBuilder object at 0x1564cfac0>>, name='context-oracle', description="This tool utilizes the EmbeddingSimilarityCalculator and SymbolSearch to provide context for a given query by computing semantic similarity between the query and all available symbols' documentation and code. The symbol with the highest combined similarity score is identified, with its source code and documentation summary forming the primary context. Additionally, if enabled, the documentation summaries of related symbols (those next most similar to the query) are included.", coroutine=None, properties={'query': {'type': 'string', 'description': 'The query string to search for.'}, 'max_additional_related_symbols': {'type': 'integer', 'description': 'The maximum number of additional related symbols to retu

DEBUG:automata.core.agent.providers:
------------------------------------------------------------
Session ID: ffcbad9c-95df-4f99-ade2-990bd018b50c
------------------------------------------------------------




In [6]:
# Run the agent
result = agent.run()

DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.core.llm.providers.openai:Approximately 2534 tokens were consumed prior to completion generation.


Function Call:
context-oracle

Arguments:
{
  "query": "SymbolRank"
}

 

DEBUG:automata.core.llm.providers.openai:Approximately 2569 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:automata.core.agent.providers:Latest User Message -- 
user:
content=Execution Result:

@property
    def symbol_rank(self):
        if self._symbol_rank is None:
            self._symbol_rank = SymbolRank(
                self.symbol_graph.default_rankable_subgraph, config=self.symbol_rank_config
            )
        return self._symbol_rank

    Documentation:

Symbol: automata.core.experimental.search.symbol_search.SymbolSearch.symbol_rank
Fetching related context now for 10 symbols...

automata.tests.unit.test_symbol_search_tool.test_symbol_rank_search

Symbol: automata.tests.unit.test_symbol_search_tool.test_symbol_rank_search


automata.tests.unit.test_symbol_rank.test_get_ranks

Symbol: a

result =  @property
    def symbol_rank(self):
        if self._symbol_rank is None:
            self._symbol_rank = SymbolRank(
                self.symbol_graph.default_rankable_subgraph, config=self.symbol_rank_config
            )
        return self._symbol_rank

    Documentation:

Symbol: automata.core.experimental.search.symbol_search.SymbolSearch.symbol_rank
Fetching related context now for 10 symbols...

automata.tests.unit.test_symbol_search_tool.test_symbol_rank_search

Symbol: automata.tests.unit.test_symbol_search_tool.test_symbol_rank_search


automata.tests.unit.test_symbol_rank.test_get_ranks

Symbol: automata.tests.unit.test_symbol_rank.test_get_ranks


automata.core.tools.builders.symbol_search.SymbolSearchToolkitBuilder._symbol_rank_search_processor

Symbol: automata.core.tools.builders.symbol_search.SymbolSearchToolkitBuilder._symbol_rank_search_processor


automata.tests.unit.test_symbol_rank.test_get_ranks_small_graph

Symbol: automata.tests.unit.test_symbol_rank

DEBUG:automata.core.llm.providers.openai:Approximately 3853 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.core.agent.providers:Latest User Message -- 
user:
content=Execution Result:

def test_exact_search(symbol_search):
    with patch(
        "automata.core.experimental.search.symbol_search.SymbolSearch._find_pattern_in_modules",
        return_value=["file1", "file2"],
    ):
        result = symbol_search.exact_search("pattern1")
        assert result == ["file1", "file2"]


Documentation:

Symbol: automata.tests.unit.test_symbol_search.test_exact_search
Fetching related context now for 10 symbols...

automata.core.tools.builders.symbol_search.SymbolSearchToolkitBuilder

The `SymbolSearchToolkitBuilder` is a key component that connects to the SymbolSearch API, allowing users to search through Python codebases. It functions by bringing t

result =  def test_exact_search(symbol_search):
    with patch(
        "automata.core.experimental.search.symbol_search.SymbolSearch._find_pattern_in_modules",
        return_value=["file1", "file2"],
    ):
        result = symbol_search.exact_search("pattern1")
        assert result == ["file1", "file2"]


Documentation:

Symbol: automata.tests.unit.test_symbol_search.test_exact_search
Fetching related context now for 10 symbols...

automata.core.tools.builders.symbol_search.SymbolSearchToolkitBuilder

The `SymbolSearchToolkitBuilder` is a key component that connects to the SymbolSearch API, allowing users to search through Python codebases. It functions by bringing together various tools for searching an indexed codebase, allowing ranking of symbols, source code retrieval and exact pattern searches. The class starts with a symbol search and a selection of optional search tools, then builds either individual tools or a complete toolkit for searching the codebase. A query can be proc

   from automata.core.experimental.search.symbol_search import SymbolSearch
   from automata.core.experimental.search.rank import SymbolRank, SymbolRankConfig
   import networkx as nx

   # Create a graph
   G = nx.DiGraph()
   G.add_edge(1, 2)
   G.add_edge(2, 3)
   G.add_edge(3, 1)

   # Initialize SymbolRankConfig
   config = SymbolRankConfig()
   
   # Initialize SymbolRank, calculate SymbolRanks, and retrieve the top-ranked symbols.
   sr = SymbolRank(G, config)
   ranks = sr.get_ranks()

   # Initialize SymbolSearch.
   ss = SymbolSearch(index_path)

   # Process a natural-language query.
   results = ss.process_query("function to calculate sum")

   # Retrieve the source code for a particular symbol.
   source = ss.retrieve_source_code_by_symbol('numpy.sum')
   ```

All in all, the `SymbolRank` and `SymbolSearch` classes form a crucial part of the Automata program that enables users to efficiently and conveniently interact and navigate through a Python codebase.Function Call:
ca

DEBUG:automata.core.llm.providers.openai:Approximately 6164 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.core.agent.providers:Latest User Message -- 
user:
content=Execution Result:

{"SymbolRank": "The SymbolRank class is designed to rank symbols within a software's semantic and structural context using the PageRank algorithm. It constructs a SymbolGraph where each node is a symbol from the application’s corpus, and the edges represent dependencies. This graph, along with a similarity dictionary, is used to calculate the SymbolRanks, each denoting the prominence of a symbol within the software. The class also utilizes a SymbolRankConfig configuration class for setting up necessary parameters for calculations, such as alpha (a damping factor), max_iterations, and tolerance. However, it could be time-consuming for large graphs due to its ite

In [7]:
# Print the result
print(f"Result:\n{result}")

Result:
Execution Result:

{"SymbolRank": "The SymbolRank class is designed to rank symbols within a software's semantic and structural context using the PageRank algorithm. It constructs a SymbolGraph where each node is a symbol from the application’s corpus, and the edges represent dependencies. This graph, along with a similarity dictionary, is used to calculate the SymbolRanks, each denoting the prominence of a symbol within the software. The class also utilizes a SymbolRankConfig configuration class for setting up necessary parameters for calculations, such as alpha (a damping factor), max_iterations, and tolerance. However, it could be time-consuming for large graphs due to its iterative algorithm, and might return inaccurate results if the symbol graph and similarity dictionary aren't properly managed.", "SymbolSearch": "The SymbolSearch class offers a convenient means of searching symbols within an indexed codebase, covering tasks such as exact pattern matching, NLP-formatted q

In [8]:
result

'Execution Result:\n\n{"SymbolRank": "The SymbolRank class is designed to rank symbols within a software\'s semantic and structural context using the PageRank algorithm. It constructs a SymbolGraph where each node is a symbol from the application’s corpus, and the edges represent dependencies. This graph, along with a similarity dictionary, is used to calculate the SymbolRanks, each denoting the prominence of a symbol within the software. The class also utilizes a SymbolRankConfig configuration class for setting up necessary parameters for calculations, such as alpha (a damping factor), max_iterations, and tolerance. However, it could be time-consuming for large graphs due to its iterative algorithm, and might return inaccurate results if the symbol graph and similarity dictionary aren\'t properly managed.", "SymbolSearch": "The SymbolSearch class offers a convenient means of searching symbols within an indexed codebase, covering tasks such as exact pattern matching, NLP-formatted quer